# Tool calling using LangChain

Many AI applications interact directly with humans. In these cases, it is appropriate for models to respond in natural language. But what about cases where we want a model to also interact directly with systems, such as databases or an API? These systems often have a particular input schema; for example, APIs frequently have a required payload structure. This need motivates the concept of tool calling. You can use tool calling to request model responses that match a particular schema.

## Key concepts

1. **Tool Creation:** Use the `@tool` decorator to create a tool. A tool is an association between a function and its schema. 
2. **Tool Binding:** The tool needs to be connected to a model that supports tool calling. This gives the model awareness of the tool and the associated input schema required by the tool.
3. **Tool Calling:** When appropriate, the model can decide to call a tool and ensure its response conforms to the tool's input schema.
4. **Tool Execution:** The tool can be executed using the arguments provided by the model.

![](images/tool_calling_components.png)

## Prerequisites

* Azure OpenAI GPT model
* Setup `.env` file with the required credentials

To install `LangChain` run:

In [2]:
%pip install -U langchain-community langgraph --quiet

Note: you may need to restart the kernel to use updated packages.


## Using Language Models

Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably.

In [5]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

## Tool creation

The recommended way to create a tool is using the `@tool` decorator.

In [6]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b."""
    return a * b

## Tool binding

The central concept to understand is that `LangChain` provides a standardized interface for connecting tools to models. The `.bind_tools()` method can be used to specify which tools are available for a model to call.

As a specific example, let's take a function multiply and bind it as a tool to a model that supports tool calling.

In [7]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = model.bind_tools([multiply])

## Tool calling

![](images/tool-calling.png)

A key principle of tool calling is that the model decides when to use a tool based on the input's relevance. The model doesn't always need to call a tool. For example, given an unrelated input, the model would not call the tool:

In [11]:
result = llm_with_tools.invoke("Hello world!")

The result would be an AIMessage containing the model's response in natural language (e.g., "Hello!"). However, if we pass an input relevant to the tool, the model should choose to call it:

In [12]:
result = llm_with_tools.invoke("What is 2 multiplied by 3?")

As before, the output result will be an `AIMessage`. But, if the tool was called, result will have a `tool_calls` attribute. This attribute includes everything needed to execute the tool, including the tool name and input arguments:

In [13]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_JpJ5kos6mkRCM7kj73AupdbE',
  'type': 'tool_call'}]

## Best practices

When designing tools to be used by a model, it is important to keep in mind that:

Models that have explicit tool-calling APIs will be better at tool calling than non-fine-tuned models.
Models will perform better if the tools have well-chosen names and descriptions.
Simple, narrowly scoped tools are easier for models to use than complex tools.
Asking the model to select from a large list of tools poses challenges for the model.

## More resources

https://python.langchain.com/docs/concepts/tool_calling/